In [ ]:
from pydub import AudioSegment
import os
import random

def extract_random_segment(audio_segment, segment_length_ms=60000):
    # Ensure the segment length is not longer than the audio duration
    if len(audio_segment) <= segment_length_ms:
        return audio_segment
    
    start_ms = random.randint(0, len(audio_segment) - segment_length_ms)
    end_ms = start_ms + segment_length_ms
    return audio_segment[start_ms:end_ms]

def process_audio_files(source_folder, destination_folder, segment_length_ms=60000):
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)
    
    # Supported audio formats and corresponding loading functions
    supported_formats = {
        '.mp3': AudioSegment.from_mp3,
        '.wav': AudioSegment.from_wav,
        '.ogg': AudioSegment.from_ogg,
        '.flv': AudioSegment.from_flv,
        '.mp4': AudioSegment.from_file,
        '.wma': AudioSegment.from_file,
        '.aac': AudioSegment.from_file,
        '.m4a': AudioSegment.from_file
    }
    
    # Iterate through all files in the source folder
    for filename in os.listdir(source_folder):
        file_extension = os.path.splitext(filename)[1].lower()
        input_file_path = os.path.join(source_folder, filename)
        
        # Check if the file format is supported
        if file_extension in supported_formats:
            try:
                # Load the audio file using the appropriate loading function
                load_audio = supported_formats[file_extension]
                if file_extension in ['.mp4', '.wma', '.aac', '.m4a']:
                    audio = load_audio(input_file_path, format=file_extension[1:])
                else:
                    audio = load_audio(input_file_path)
                
                # Extract a random segment
                segment = extract_random_segment(audio, segment_length_ms)
                
                # Save the segment to the destination folder in WAV format
                segment_filename = f"{os.path.splitext(filename)[0]}_random_segment.wav"
                segment_file_path = os.path.join(destination_folder, segment_filename)
                segment.export(segment_file_path, format="wav")
                print(f"Saved random segment of {filename} to {segment_file_path}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        else:
            print(f"Unsupported file format for {filename}, skipping.")

# Example usage
source_folder_path = r"D:\new_data\neg_data\_background_noise_\original"
destination_folder_path =r"D:\new_data\neg_data\_background_noise_\1_minute"
process_audio_files(source_folder_path, destination_folder_path, segment_length_ms=60000)


In [ ]:
# extracting chunks
from pydub import AudioSegment
import os

def split_audio(audio_segment, chunk_length_ms=2000):
    chunks = []
    for i in range(0, len(audio_segment), chunk_length_ms):
        chunk = audio_segment[i:i + chunk_length_ms]
        chunks.append(chunk)
    return chunks

def process_audio_files(source_folder, destination_folder, chunk_length_ms=2000):
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)
    
    # Supported audio formats and corresponding loading functions
    supported_formats = {
        '.mp3': AudioSegment.from_mp3,
        '.wav': AudioSegment.from_wav,
        '.ogg': AudioSegment.from_ogg,
        '.flv': AudioSegment.from_flv,
        '.mp4': AudioSegment.from_file,  # Can use from_file with explicit format
        '.wma': AudioSegment.from_file,  # Example; requires ffmpeg for some formats
        '.aac': AudioSegment.from_file,
        '.m4a': AudioSegment.from_file
    }
    
    # Iterate through all files in the source folder
    for filename in os.listdir(source_folder):
        file_extension = os.path.splitext(filename)[1].lower()
        input_file_path = os.path.join(source_folder, filename)
        
        # Check if the file format is supported
        if file_extension in supported_formats:
            try:
                # Load the audio file using the appropriate loading function
                load_audio = supported_formats[file_extension]
                if file_extension in ['.mp4', '.wma', '.aac', '.m4a']:
                    audio = load_audio(input_file_path, format=file_extension[1:])
                else:
                    audio = load_audio(input_file_path)
                
                # Split the audio into chunks
                chunks = split_audio(audio, chunk_length_ms)
                
                # Save each chunk to the destination folder in WAV format
                for i, chunk in enumerate(chunks):
                    chunk_filename = f"{os.path.splitext(filename)[0]}_chunk{i}.wav"
                    chunk_file_path = os.path.join(destination_folder, chunk_filename)
                    chunk.export(chunk_file_path, format="wav")  # Save as WAV
                    print(f"Saved chunk {i} of {filename} to {chunk_file_path}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        else:
            print(f"Unsupported file format for {filename}, skipping.")

# Example usage
source_folder_path =  r"D:\new_data\neg_data\silence"
destination_folder_path = r"D:\new_data\neg_data\silence\chunks\5s_chunks"
process_audio_files(source_folder_path, destination_folder_path, chunk_length_ms=5000)


In [ ]:
import wave
import pandas as pd
import os
def get_sample_rate(audio_path):
    with wave.open(audio_path, 'rb') as wav_file:
        sample_rate = wav_file.getframerate()
        num_frames = wav_file.getnframes()
        duration = num_frames / sample_rate
    return sample_rate,duration,audio_path

folder =  r"D:\new_data\hilfe_hilfe\processed\1\cleaning\3.1 clean"

audios = os.listdir(folder)
audio_paths = [os.path.join(folder, file_name) for file_name in audios]
audio_info = [get_sample_rate(a) for a in audio_paths]
df = pd.DataFrame(audio_info, columns =['sample_rate', 'duration','audio_path'])
min_duration = df['duration'].quantile(0.01)
max_duration = df['duration'].quantile(0.99)
print(min_duration,max_duration)
new_df = df.loc[(df['duration']>=min_duration) & (df['duration']<=max_duration)]
new_df.describe()

In [ ]:
import shutil
destination_folder = '/'.join([*(folder.split('\\')[:-1]),'percentile'])
for pth in new_df['audio_path']:
    destination_file = f"{destination_folder}/{pth.split('\\')[-1]}"
    shutil.copy2(pth,destination_file)
    

In [ ]:
import librosa
from plot_audios import plot_mel_spectrogram
import shutil
import os
from IPython.display import clear_output, Audio, display

word = 'hilfe'
folder = f"D:/new_data/{word}/processed/1/preprocessing/3.1 clean"
ignored = f"D:/new_data/{word}/processed/1/preprocessing/3.2 ignored"
folder, ignored = ignored, folder
files_moved = list()
for f in os.listdir(folder):
    audio_file = f'{os.path.join(folder,f)}'
    y, sr = librosa.load(audio_file,sr=None)
    plot_mel_spectrogram(y, sr, f)
    display(Audio(y, rate=sr, autoplay=True))
    ignore = input('move to ignore? ')
    if ignore.lower() in ['y','yes']:
        ignore_file = f'{os.path.join(ignored,f)}'
        x = shutil.move(audio_file,ignore_file)
        files_moved.append(x)
    clear_output(wait=True)

In [2]:
# remove silence
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def remove(input_file, output_file, min_silence_len=200, silence_thresh=-45):
    audio = AudioSegment.from_file(input_file)
    chunks = split_on_silence(audio, min_silence_len=min_silence_len,silence_thresh=silence_thresh)
    output = AudioSegment.empty()
    for chunk in chunks:
        output += chunk
    output.export(output_file,format='wav')


def process_audio(source_folder,destination_folder,audio_name):
    for idx,file_name in enumerate(os.listdir(source_folder)):
        f = f'{audio_name}_{idx}.wav'
        source_file_path = os.path.join(source_folder, file_name)
        destination_file_path = os.path.join(destination_folder, f)
        remove(source_file_path,destination_file_path)


source_folder = r"D:\new_data\adele\processed\2\cleaning\clean"
destination_folder = r"D:\new_data\adele\processed\2\cleaning\trimmed"
process_audio(source_folder,destination_folder,'Adele')

In [1]:
import os
neg_data_archive = r'D:\data_code\data\words'
word_folders = os.listdir(neg_data_archive)
for i in word_folders:
    files = os.listdir(f'{neg_data_archive}/{i}')
    for idx,j in enumerate(files):
        file_path = f'{neg_data_archive}/{i}/{j}'
        new_path = f'{neg_data_archive}/{i}/{i}_{idx}.wav'
        os.rename(file_path,new_path)

In [2]:
# files to single file
import os
import shutil
src_folder = r'D:\data_code\data\words'
dst_folder = r'D:\data_code\data\all_words'
word_folders = os.listdir(src_folder)
for i in word_folders:
    files = os.listdir(f'{src_folder}/{i}')
    for idx,j in enumerate(files):
        src_file = f'{src_folder}/{i}/{j}'
        dst_file = f'{dst_folder}/{j}'
        shutil.copy2(src_file,dst_file)

In [ ]:
# overlay
import glob
import pydub
import random

def overlay(source_folder,bg_folder,destinatio_folder):
    bg_files = glob.glob(f'{bg_folder}*')
    files = glob.glob(f'{source_folder}*')
    for file in files:
        f = file.replace('\\','/').split('/')[-1]
        bg_file = random.choice(bg_files)
        bg_audio_segment = pydub.AudioSegment.from_file(bg_file)
        bg_audio_length = len(bg_audio_segment)
        bg_audio_mid = bg_audio_length // 2

        for i in range(2):
            
            if i == 0:
                audio_segment = pydub.AudioSegment.from_file(file)
                start_position = random.randint(0,bg_audio_mid)
                overlay_audio = bg_audio_segment.overlay(audio_segment,position=start_position)
            else:
                audio_segment = pydub.AudioSegment.from_file(random.choice(files))
                start_position = random.randint(bg_audio_mid,bg_audio_length)
                overlay_audio = overlay_audio.overlay(audio_segment,position=start_position)
        
        dest_file = f'{destinatio_folder}/3_{f}'
        overlay_audio.export(dest_file,format='wav')

source_folder = r'D:\data_code\data\neg_data\speech_command\all_words\original/'
destinatio_folder = r'D:\data_code\data\neg_data\speech_command\all_words\5_seconds/'
bg_folder = r'D:\data_code\data\neg_data\silence\chunks\5s_chunks/'
overlay(source_folder,bg_folder,destinatio_folder)

In [5]:
# different files to one file
import shutil
import glob
import os 

def all_to_one(source,destination):
    files = glob.glob(f'{source}*')
    for file in files:
        f = file.replace('\\','/').split('/')[-1]
        folder = f.split('_')[1]
        dest_folder = f'{destination}{folder}'
        os.makedirs(dest_folder,exist_ok=True)
        shutil.copy2(file, f'{dest_folder}/{f}')

source_folder = r'D:\data_code\data\neg_data\speech_command\all_words\3_seconds/'
destination_folder = r'D:\data_code\data\neg_data\speech_command\words\3_seconds/'
all_to_one(source_folder,destination_folder)